In [1]:
from pymongo import MongoClient, ASCENDING
from pymongo.server_api import ServerApi
from datetime import datetime
import json
from typing import Optional, List, Dict, Any
import pandas as pd

import urllib3
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from io import StringIO
import json
from headers import *
import controllers
import models
import importlib

importlib.reload(controllers)
importlib.reload(models)
controller = controllers.FuturesDataController
model = models.FuturesDataModel()

In [2]:
client = model.connect_mongo()

成功連接到 MongoDB!


In [3]:
db = client['findata']
collection = db['twse_index']

In [4]:
date = "2024/09/09"
df_data = controller.crawler_twse_data(date)
df_data2 =controller.crawler_twse_total_return(date)

if df_data is None or df_data.empty:
    print("data1無法獲取證交所數據或數據為空")
if df_data2 is None or df_data2.empty:
    print("data2無法獲取證交所數據或數據為空")
merged_df = pd.merge(df_data,df_data2,on='date',how='outer')
merged_df

爬取日期: 20240101


,date,open,high,low,close,tri_close
0,2024-09-02,22341.9,22439.31,22178.04,22235.1,48960.74
1,2024-09-03,22240.11,22303.5,22092.21,22092.21,48647.56
2,2024-09-04,21574.65,21574.65,20922.17,21092.75,46447.84
3,2024-09-05,21271.09,21488.81,21150.99,21187.71,46663.02
4,2024-09-06,21255.45,21461.71,21125.14,21435.19,47208.33
5,2024-09-09,21127.81,21163.91,20922.51,21144.44,46568.93
6,2024-09-10,21252.57,21324.43,20977.0,21064.08,46391.97


In [7]:
existing_data = collection.find_one({'date':date})
print(existing_data)

None


In [8]:
data_dict = merged_df.to_dict('records')
data_dict

[{'date': Timestamp('2024-09-02 00:00:00'),
  'open': 22341.9,
  'high': 22439.31,
  'low': 22178.04,
  'close': 22235.1,
  'tri_close': 48960.74},
 {'date': Timestamp('2024-09-03 00:00:00'),
  'open': 22240.11,
  'high': 22303.5,
  'low': 22092.21,
  'close': 22092.21,
  'tri_close': 48647.56},
 {'date': Timestamp('2024-09-04 00:00:00'),
  'open': 21574.65,
  'high': 21574.65,
  'low': 20922.17,
  'close': 21092.75,
  'tri_close': 46447.84},
 {'date': Timestamp('2024-09-05 00:00:00'),
  'open': 21271.09,
  'high': 21488.81,
  'low': 21150.99,
  'close': 21187.71,
  'tri_close': 46663.02},
 {'date': Timestamp('2024-09-06 00:00:00'),
  'open': 21255.45,
  'high': 21461.71,
  'low': 21125.14,
  'close': 21435.19,
  'tri_close': 47208.33},
 {'date': Timestamp('2024-09-09 00:00:00'),
  'open': 21127.81,
  'high': 21163.91,
  'low': 20922.51,
  'close': 21144.44,
  'tri_close': 46568.93},
 {'date': Timestamp('2024-09-10 00:00:00'),
  'open': 21252.57,
  'high': 21324.43,
  'low': 20977.0,
 

In [10]:
for record in data_dict:
    record['date'] = datetime.strptime(date,"%Y/%m/%d")
data_dict

[{'date': datetime.datetime(2024, 9, 9, 0, 0),
  'open': 22341.9,
  'high': 22439.31,
  'low': 22178.04,
  'close': 22235.1,
  'tri_close': 48960.74},
 {'date': datetime.datetime(2024, 9, 9, 0, 0),
  'open': 22240.11,
  'high': 22303.5,
  'low': 22092.21,
  'close': 22092.21,
  'tri_close': 48647.56},
 {'date': datetime.datetime(2024, 9, 9, 0, 0),
  'open': 21574.65,
  'high': 21574.65,
  'low': 20922.17,
  'close': 21092.75,
  'tri_close': 46447.84},
 {'date': datetime.datetime(2024, 9, 9, 0, 0),
  'open': 21271.09,
  'high': 21488.81,
  'low': 21150.99,
  'close': 21187.71,
  'tri_close': 46663.02},
 {'date': datetime.datetime(2024, 9, 9, 0, 0),
  'open': 21255.45,
  'high': 21461.71,
  'low': 21125.14,
  'close': 21435.19,
  'tri_close': 47208.33},
 {'date': datetime.datetime(2024, 9, 9, 0, 0),
  'open': 21127.81,
  'high': 21163.91,
  'low': 20922.51,
  'close': 21144.44,
  'tri_close': 46568.93},
 {'date': datetime.datetime(2024, 9, 9, 0, 0),
  'open': 21252.57,
  'high': 21324.43

In [11]:
result = collection.insert_many(data_dict)